In [0]:
import os, pickle, re, sklearn, string
import numpy as np
import matplotlib.pyplot as plt

from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split




In [18]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')# Download text data sets, including stop words
from nltk.corpus import stopwords # Import the stop word list
import re
from bs4 import BeautifulSoup


# Download the punkt tokenizer for sentence splitting
import nltk.data
nltk.download('punkt')   

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')


from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords


class Creat_vocabulary():
  
  def __init__(self):
    self.sentences = []
    self.word2idx = {}
    self.idx2word = {}
    self.idx_pairs = []
    self.all_voc = []
    
    
  def get_sentence(self):
    
      return self.sentences
    
    
  def get_word2idx(self):
    
      return self.word2idx
  
  
  def get_idx2word(self):
    
      return self.idx2word
  
  
  def get_idx_pairs(self):
    
      return self.idx_pairs
    
  def expand(self,ListofList):
    
    for ll in ListofList:
      for l in ll:
        self.all_voc.append(l)
    return self.all_voc   
  

  # Define a function to split a review into parsed sentences
  def review_to_sentences( self, review, tokenizer, remove_stopwords=False ):
      # Function to split a review into parsed sentences. Returns a 
      # list of sentences, where each sentence is a list of words
    
      # 1. Use the NLTK tokenizer to split the paragraph into sentences
      raw_sentences = tokenizer.tokenize(review.strip())
    
    
      # 2. Loop over each sentence
      sentences = []
      for raw_sentence in raw_sentences:
      
          # If a sentence is empty, skip it
          if len(raw_sentence) > 0:
          
              # Otherwise, call review_to_wordlist to get a list of words
              sentences.append( self.review_to_wordlist( raw_sentence,remove_stopwords ))
    
      # Return List os List of Tokens in a sentence
      return sentences

  
  def review_to_wordlist(self, review, remove_stopwords=False ):
  
    # Function to convert a document to a sequence of words,
    # optionally removing stop words. Returns a list of words.

    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()

    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)

    # 3. Convert words to lower case and split them
    words = review_text.lower().split()

    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]

    # 5. Return a list of words
    return(words)


  def reviews_to_Words(self,review, tokenizer):
  
    sentences = []  # Initialize an empty list of sentences

    print ("Parsing sentences from training set")
  
    for review in data1["review"]:
      self.sentences += self.review_to_sentences(review, tokenizer)
  
    return self.sentences
  
 
  
  def dictionary(self,review, tokenizer ):
    
    vocabulary = []
    for sentence in self.sentences:
      for token in sentence:
          if token not in vocabulary:
              vocabulary.append(token)

    self.word2idx = {w: idx for (idx, w) in enumerate(vocabulary)}
    self.idx2word = {idx: w for (idx, w) in enumerate(vocabulary)}

    vocabulary_size = len(vocabulary)
    
    return vocabulary_size, self.word2idx, self.idx2word
  
  def skip_contex_tuple(self,context_size):
    
    
    # changing context_size to window size
    if(context_size % 2 != 0):
      window_size = int(context_size + 1)//2
    else:
      window_size = int(context_size)//2
  
    #find context words around each word in a sentence and then coolect it in a list using vocab index in our dic
    for sentence in self.sentences:
      
      
      indices = [self.word2idx[word] for word in sentence]
      
      # for each word, threated as center word
      for center_word_pos in range(len(indices)):
        tupe =[]
        
        for w in range(-window_size, window_size):
          cntx_pos = center_word_pos + w 
          

          if cntx_pos <=0 or cntx_pos>=len(indices) or cntx_pos == center_word_pos:

            continue
            
          tup.append(sentence[cntx_pos])
            
        # for each window position
        #tup = [sentence[center_word_pos + w ] for w in range(-window_size, window_size) if center_word_pos + w > 0 and center_word_pos + w <= len(indices) and center_word_pos  != center_word_pos + w ]

        self.idx_pairs.append((indices[center_word_pos], tup))

    self.idx_pairs = np.array(self.idx_pairs) # it will be useful to have this as numpy array
    
    return self.idx_pairs

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:

import random

class RandomGenerator(object):
    """Draw a random int in [0, n] according to n sampling weights"""
    def __init__(self, sampling_weights):
        self.population = list(range(len(sampling_weights)))
        self.sampling_weights = sampling_weights
        self.candidates = []
        self.i = 0

    def draw(self):
        if self.i == len(self.candidates):
            self.candidates = random.choices(
                self.population, self.sampling_weights, k=10000)
            self.i = 0
        self.i += 1
        return self.candidates[self.i-1]

      
      
      

generator = RandomGenerator([2,3,4])
[generator.draw() for _ in range(10)]

[1, 0, 0, 1, 2, 0, 2, 2, 0, 2]

In [0]:




def get_negatives(all_contexts, sampling_weights, K):
    all_negatives = []
    generator = RandomGenerator(sampling_weights)
    for contexts in all_contexts:
        negatives = []
        while len(negatives) < len(contexts) * K:
            neg = generator.draw()
            # Noise words cannot be context words
            if neg not in contexts:
                negatives.append(neg)
        all_negatives.append(negatives)
    return all_negatives

#all_negatives = get_negatives(all_contexts, sampling_weights, 5)

In [8]:
from google.colab import drive
import glob
import os
import pandas as pd

#in order to use the dataset from MyDrive, need to mount it in google colab
drive.mount('/content/gdrive/')
path = '/content/gdrive/My Drive/data/labeledTrainData.tsv'

data1 = pd.read_csv(path, header=0, delimiter="\t", quoting=3)
data = data1['review']
target = data1['sentiment']

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

# create training and testing vars
train, test, ytrain, ytest = train_test_split(data, target, test_size=0.2)
#print X_train.shape, y_train.shape
#print X_test.shape, y_test.shape

In [0]:
xtrain = train.values.reshape(-1,1)

In [19]:
sentences = []  # Initialize an empty list of sentences
vocab =  Creat_vocabulary()
sentences =vocab.reviews_to_Words(data1 , tokenizer)

vocabulary_size, word2idx, idx2word = vocab.dictionary(data1 , tokenizer )

Parsing sentences from training set


/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


In [0]:
a = sentences[2]

In [0]:
a[1]

'is'

In [0]:
idx_pairs = vocab.skip_contex_tuple(3)

In [41]:
idx_pairs

array(['1', 'this'], dtype='<U44')

In [0]:
import collections

listofV= vocab.get_sentence()

counter = collections.Counter(vocab.expand(listofV))


counters = list(counter.values())


sampling_weights = [counters[i]**0.75 for i in range(len(counters))]



[10179.149986629658,
 6442.583421550465,
 15300.320315310373,
 670.6232035533772,
 1713.5423017908186,
 1595.9126864960185,
 6348.439352855337,
 46732.0002073971,
 643.8817172917035,
 48.11478356161978,
 17032.428661556984,
 2042.210758431224,
 578.0257935375433,
 169.08641184955292,
 23641.595938853312,
 7501.435384208449,
 1374.3347117925478,
 1868.5651057710975,
 396.20480825679044,
 573.5181661851256,
 2212.501031229346,
 27263.6966935784,
 5380.615446485757,
 1087.254847573719,
 18.80301546543197,
 36.25650476828122,
 1684.3148896058485,
 1124.965998458558,
 5145.624375062926,
 1587.2406859811954,
 3169.127879139181,
 27142.125456368183,
 485.9006971642849,
 169.08641184955292,
 3117.672769872219,
 1367.245569732833,
 5923.801661602249,
 1500.9359241722602,
 10876.753000091094,
 3770.22769131538,
 581.6234734569633,
 17944.640382636197,
 106.52916761335055,
 2834.5237316004177,
 4139.003053104851,
 3950.4751297154307,
 998.1244134512888,
 529.1549057153204,
 7649.413960056906,
 19

In [0]:
import torch
from torch import nn
import torch.nn.functional as F
from torch import optim


context_size = 3
embedding_dim = 126

model = nn.Sequential(
    
    
                      nn.Embedding(vocabulary_size, embedding_dim),
                      nn.Linear(embedding_dim,256),
                      nn.Linear(256,context_size * vocabulary_size),
                      nn.LogSoftmax(dim=1))



objective = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.003)


losses= []
epoch = 4

for e in range(epoch):
  
  running_loss = 0

  for images, labels in idx_pairs:
    for context_idxs in 
      
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)
        optimizer.zero_grad()

        output = model(context_idxs)
        
        loss = objective(output, torch.tensor([word_to_ix[target]], dtype=torch.long))
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        losses.append(loss.item())
  else:
        print(f"Training loss: {running_loss/len(trainloader)}")

SyntaxError: ignored

In [0]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',    level=logging.INFO)

def myhashfxn(obj):
    return hash(obj) % (2 ** 32)

# Set values for various parameters
num_features = 400    # Word vector dimensionality                      
min_word_count = 60   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words


#**********************************************************



# Initialize and train the model (this will take some time)
from gensim.models import word2vec
#model = word2vec.Word2Vec(hashfxn=myhashfxn)

print ("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers,             size=num_features, min_count = min_word_count,             window = context, sample = downsampling,hashfxn=myhashfxn)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)


2019-06-06 02:15:44,719 : INFO : 'pattern' package not found; tag filters are not available for English
2019-06-06 02:15:44,729 : INFO : collecting all words and their counts
2019-06-06 02:15:44,730 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-06-06 02:15:44,786 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types
2019-06-06 02:15:44,845 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types
2019-06-06 02:15:44,901 : INFO : PROGRESS: at sentence #30000, processed 671315 words, keeping 30034 word types


Training model...


2019-06-06 02:15:44,963 : INFO : PROGRESS: at sentence #40000, processed 897815 words, keeping 34348 word types
2019-06-06 02:15:45,019 : INFO : PROGRESS: at sentence #50000, processed 1116963 words, keeping 37761 word types
2019-06-06 02:15:45,073 : INFO : PROGRESS: at sentence #60000, processed 1338404 words, keeping 40723 word types
2019-06-06 02:15:45,127 : INFO : PROGRESS: at sentence #70000, processed 1561580 words, keeping 43333 word types
2019-06-06 02:15:45,181 : INFO : PROGRESS: at sentence #80000, processed 1780887 words, keeping 45714 word types
2019-06-06 02:15:45,235 : INFO : PROGRESS: at sentence #90000, processed 2004996 words, keeping 48135 word types
2019-06-06 02:15:45,289 : INFO : PROGRESS: at sentence #100000, processed 2226966 words, keeping 50207 word types
2019-06-06 02:15:45,342 : INFO : PROGRESS: at sentence #110000, processed 2446580 words, keeping 52081 word types
2019-06-06 02:15:45,397 : INFO : PROGRESS: at sentence #120000, processed 2668775 words, keepin

In [0]:
model.doesnt_match("man woman child kitchen".split())


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'kitchen'

In [0]:

model.most_similar("man")



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('woman', 0.6654655933380127),
 ('doctor', 0.6339104175567627),
 ('boy', 0.6286064982414246),
 ('journalist', 0.594315767288208),
 ('businessman', 0.5905495285987854),
 ('lady', 0.5899538397789001),
 ('guy', 0.5748952627182007),
 ('soldier', 0.5712598562240601),
 ('scientist', 0.5709996223449707),
 ('priest', 0.5663358569145203)]

In [0]:
#Vector Averaging
import numpy as np # Make sure that numpy is imported
def makeFeatureVec(words, model, num_features):
# Function to average all of the word vectors in a given
# paragraph
#
# Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
#
    nwords = 0.
#
# Index2word is a list that contains the names of the words in
# the model's vocabulary. Convert it to a set, for speed
    index2word_set = set(model.wv.index2word)
#
# Loop over each word in the review and, if it is in the model's
# vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set:
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
#
# Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec



In [0]:
def getAvgFeatureVecs(reviews, model, num_features):
# Given a set of reviews (each one a list of words), calculate
# the average feature vector for each one and return a 2D numpy array
#
# Initialize a counter
    counter = 0
#
# Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
# Loop through the reviews
    for review in reviews:
        if counter%1000 == 0:
            print ("Review %d of %d" % (counter, len(reviews)))
        reviewFeatureVecs[counter] = makeFeatureVec(review, model,num_features)
#
# Increment the counter
        counter = counter + 1
    return reviewFeatureVecs




In [0]:
clean_train_reviews = []
for review in train1:
    clean_train_reviews.append( review_to_wordlist( review,         remove_stopwords=True ))
    
trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )
print ("Creating average feature vecs for test reviews")
clean_test_reviews = []
for review in test1:
    clean_test_reviews.append( review_to_wordlist( review,         remove_stopwords=True ))


# In[52]:

NameError: ignored

In [0]:
def expand(lll):
  s = []
  for ll in lll:
    for l in ll:
      s.append(l)
  return s   

In [0]:
l = [[1],[2,3],[4],[5],[5,5],[5],[5]]

q = expand(l)


In [29]:
print(q)

[1, 2, 3, 4, 5, 5, 5, 5, 5]


In [30]:
l = [[1],[2,3],[4],[5],[5,5],[5],[5]]
q = expand(l)

collections.Counter(q)

Counter({1: 1, 2: 1, 3: 1, 4: 1, 5: 5})